In [1]:
!pip3 install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.5.1-py3-none-any.whl size=17126 sha256=cc3a3947036e03a26714eb57ca767583e73842d85e4343284f4f6939ba3f2f76
  Stored in directory: /home/sahil/.cache/pip/wheels/bd/da/5f/2e65e230a994adb965df002373975e66b3bd1cb90c0706b9ac
Successfully built pandarallel


In [6]:
%config Completer.use_jedi = False
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# pd.set_option('display.max_colwidth', None)
from sklearn.model_selection import *
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import roc_auc_score as auc, f1_score
import lightgbm as lgb
import gc
from pandarallel import pandarallel
pandarallel.initialize()
from tqdm.notebook import tqdm
pd.options.display.max_columns = 100

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [9]:
%%time
train_bureau = pd.read_csv(DATA_DIR+'train_bureau.csv')
test_bureau = pd.read_csv(DATA_DIR+'test_bureau.csv')
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')
# data_dict = pd.read_csv('data_dict.csv')

CPU times: user 3.12 s, sys: 212 ms, total: 3.33 s
Wall time: 3.75 s


In [13]:
pd.read_csv('train_Data.csv').columns

Index(['ID', 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode',
       'BranchID', 'Area', 'Tenure', 'AssetCost', 'AmountFinance',
       'DisbursalAmount', 'EMI', 'DisbursalDate', 'MaturityDAte', 'AuthDate',
       'AssetID', 'ManufacturerID', 'SupplierID', 'LTV', 'SEX', 'AGE',
       'MonthlyIncome', 'City', 'State', 'ZiPCODE', 'Top-up Month'],
      dtype='object')

In [11]:
# ss = pd.read_csv('sample_submission_ejm25Dc.csv')

ID_COL, TARGET_COL = 'ID', 'Top-up Month'
target_mapper = pd.Series({'No Top-up Service': 0,
                           '12-18 Months': 1,
                           '18-24 Months': 2,
                           '24-30 Months': 3,
                           '30-36 Months': 4,
                           '36-48 Months': 5,
                           ' > 48 Months': 6,
 })
target_inv_mapper = pd.Series(index = target_mapper.values, data = target_mapper.index)
train[TARGET_COL] = train[TARGET_COL].map(target_mapper)

In [12]:
df = train.append(test).reset_index(drop = True)
date_cols = ['DisbursalDate', 'MaturityDAte', 'AuthDate']
df[date_cols] = df[date_cols].apply(lambda x: pd.to_datetime(x))

del df['AssetID']
gc.collect()

414

In [13]:
orig_columns = df.columns.tolist()
orig_fts = [c for c in df.columns if c not in ['ID', 'Top-up Month']]

In [14]:
df[[c + '_freq' for c in orig_fts]] = df[orig_fts].apply(lambda x: x.map(x.value_counts()))

In [15]:
df_bureau = train_bureau.append(test_bureau).reset_index(drop = True)
df_bureau = df_bureau.drop_duplicates(['ID','DISBURSED-DT'],keep='first')
date_cols = ['DATE-REPORTED', 'DISBURSED-DT', 'CLOSE-DT', 'LAST-PAYMENT-DATE']
df_bureau[date_cols] = df_bureau[date_cols].apply(lambda x: pd.to_datetime(x, errors = 'coerce'))
df_bureau = df_bureau.sort_values(by = ['ID', 'DISBURSED-DT']).reset_index(drop = True)
df_bureau['app_dd'] = df_bureau['ID'].map(df.set_index('ID')['DisbursalDate'])

In [16]:
fltr = df_bureau['CLOSE-DT'].isnull()
df_bureau.loc[fltr, 'CLOSE-DT'] = df_bureau.loc[fltr, 'LAST-PAYMENT-DATE']

In [17]:
df_bureau['accounts_closed'] = (~df_bureau['CLOSE-DT'].isnull()) * 1
df_bureau['total_accounts_closed'] = df_bureau.groupby("ID")['accounts_closed'].transform('sum')
df_bureau['avg_no_accounts_closed'] = df_bureau.groupby("ID")['accounts_closed'].transform('mean')
df_bureau['total_accounts_closed_ACCT-TYPE'] = df_bureau.groupby(["ID", 'ACCT-TYPE'])['accounts_closed'].transform('sum')
df_bureau['avg_no_accounts_closed_ACCT-TYPE'] = df_bureau.groupby(["ID", 'ACCT-TYPE'])['accounts_closed'].transform('mean')
df_bureau['total_accounts_closed_CONTRIBUTOR-TYPE'] = df_bureau.groupby(["ID", 'CONTRIBUTOR-TYPE'])['accounts_closed'].transform('sum')
df_bureau['avg_no_accounts_closed_CONTRIBUTOR-TYPE'] = df_bureau.groupby(["ID", 'CONTRIBUTOR-TYPE'])['accounts_closed'].transform('mean')

In [18]:
fltr = df_bureau['app_dd'] == df_bureau['DISBURSED-DT']
for f in ['total_accounts_closed_ACCT-TYPE', 'avg_no_accounts_closed_ACCT-TYPE', 'total_accounts_closed_CONTRIBUTOR-TYPE',
         'avg_no_accounts_closed_CONTRIBUTOR-TYPE']:
    df[f] = df['ID'].map(df_bureau[fltr].set_index('ID')[f])

In [19]:
df_bureau['ba'] = pd.factorize(df_bureau['ACCT-TYPE'].astype('str') + df_bureau['CONTRIBUTOR-TYPE'].astype('str'))[0]

In [20]:
def clean(x):
    try:
        return float(str(x).replace(',', ''))
    except:
        return np.nan

In [21]:
df_bureau['CURRENT-BAL'] = df_bureau['CURRENT-BAL'].apply(lambda x: clean(x))
df_bureau['INSTALLMENT-AMT'] = df_bureau['INSTALLMENT-AMT'].apply(lambda x: clean(x))
df_bureau['CREDIT-LIMIT/SANC AMT'] = df_bureau['CREDIT-LIMIT/SANC AMT'].apply(lambda x: clean(x))

In [22]:
for c in tqdm(['DATE-REPORTED', 'DISBURSED-DT', 'CLOSE-DT']):
    df_bureau[c + '_days_since_start'] = (df_bureau[c] - df_bureau[c].min()).dt.days
    df_bureau[c + '_days_from_end'] = (df_bureau[c].max() - df_bureau[c]).dt.days
    
    df_bureau[c + '_ACCT_TYPE_min'] = df_bureau.groupby('ACCT-TYPE')[c + '_days_since_start'].transform('min')
    df_bureau[c + '_ACCT_TYPE_max'] = df_bureau.groupby('ACCT-TYPE')[c + '_days_since_start'].transform('max')
    
    df_bureau[c + '_max'] = df_bureau.groupby('ID')[c].transform('max')
    df_bureau[c + '_min'] = df_bureau.groupby('ID')[c].transform('min')
    df_bureau[c + '_2nd_max'] = df_bureau.groupby('ID')[c].transform('nth', -2)
    df_bureau[c + '_2nd_min'] = df_bureau.groupby('ID')[c].transform('nth', 1)
    
    df_bureau[c + '_range_in_days'] = (df_bureau[c + '_max'] - df_bureau[c + '_min']).dt.days
    
    df_bureau[c + '_next'] = df_bureau.groupby('ID')[c].shift(-1)
    df_bureau[c + '_prev'] = df_bureau.groupby('ID')[c].shift(1)
    
    df_bureau[c + '_2nd_next'] = df_bureau.groupby('ID')[c].shift(-2)
    df_bureau[c + '_2nd_prev'] = df_bureau.groupby('ID')[c].shift(2)
    
    df_bureau[c + '_diff_to_next'] = (df_bureau[c + '_next'] - df_bureau[c]).dt.days
    df_bureau[c + '_diff_to_prev'] = (df_bureau[c] - df_bureau[c + '_prev']).dt.days
    df_bureau[c + '_diff_prev_to_next'] = (df_bureau[c + '_next'] - df_bureau[c + '_prev']).dt.days
    df_bureau[c + '_diff_to_2nd_next'] = (df_bureau[c + '_2nd_next'] - df_bureau[c]).dt.days
    df_bureau[c + '_diff_to_2nd_prev'] = (df_bureau[c] - df_bureau[c + '_2nd_prev']).dt.days
    
    df_bureau[c + '_diff_to_max'] = (df_bureau[c + '_max'] - df_bureau[c]).dt.days
    df_bureau[c + '_diff_to_min'] = (df_bureau[c] - df_bureau[c + '_min']).dt.days
    df_bureau[c + '_diff_to_2nd_max'] = (df_bureau[c + '_2nd_max'] - df_bureau[c]).dt.days
    df_bureau[c + '_diff_to_2nd_min'] = (df_bureau[c] - df_bureau[c + '_2nd_min']).dt.days
    
    df_bureau[c+ '_diff_to_next_acct_type'] = (df_bureau[c] - df_bureau.groupby(['ACCT-TYPE', 'ID'])[c].shift(-1)).dt.days
    df_bureau[c+ '_diff_to_prev_acct_type'] = (df_bureau[c] - df_bureau.groupby(['ACCT-TYPE', 'ID'])[c].shift(1)).dt.days
    
    df_bureau[c+ '_diff_to_next_CONTRIBUTOR-TYPE'] = (df_bureau[c] - df_bureau.groupby(['CONTRIBUTOR-TYPE', 'ID'])[c].shift(-1)).dt.days
    df_bureau[c+ '_diff_to_prev_CONTRIBUTOR-TYPE'] = (df_bureau[c] - df_bureau.groupby(['CONTRIBUTOR-TYPE', 'ID'])[c].shift(1)).dt.days
    
    df_bureau[f'max_{c}_ACCT-TYPE'] = df_bureau.groupby(['ACCT-TYPE', 'ID'])[c].transform('max')
    df_bureau[f'min_{c}_ACCT-TYPE'] = df_bureau.groupby(['ACCT-TYPE', 'ID'])[c].transform('min')
    df_bureau[f'range_{c}_ACCT_TYPE'] = (df_bureau[f'max_{c}_ACCT-TYPE'] - df_bureau[f'min_{c}_ACCT-TYPE']).dt.days
    df_bureau[f'diff_to_max_{c}_ACCT-TYPE'] = (df_bureau[c] - df_bureau[f'max_{c}_ACCT-TYPE']).dt.days
    df_bureau[f'diff_to_min_{c}_ACCT-TYPE'] = (df_bureau[c] - df_bureau[f'min_{c}_ACCT-TYPE']).dt.days
    
    fltr = df_bureau['app_dd'] == df_bureau['DISBURSED-DT']
    for f in [c + '_diff_to_next', c + '_diff_to_prev', c + '_diff_prev_to_next', c + '_range_in_days',
             c + '_diff_to_max', c + '_diff_to_min', c + '_diff_to_2nd_max', c + '_diff_to_2nd_min',
             c + '_diff_to_2nd_next', c + '_diff_to_2nd_prev',c+ '_diff_to_next_acct_type', c+ '_diff_to_prev_acct_type'
              , f'diff_to_max_{c}_ACCT-TYPE',f'diff_to_min_{c}_ACCT-TYPE', c+ '_diff_to_next_CONTRIBUTOR-TYPE',
             c+ '_diff_to_prev_CONTRIBUTOR-TYPE', 'total_accounts_closed','avg_no_accounts_closed', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE',
             c + '_days_since_start', c + '_days_from_end', c + '_ACCT_TYPE_max', c + '_ACCT_TYPE_min']:
        df[f] = df['ID'].map(df_bureau[fltr].set_index('ID')[f])

df['ID_freq'] = df['ID'].map(df_bureau['ID'].value_counts())

for c in tqdm(['DATE-REPORTED', 'CLOSE-DT']):
    df_bureau[f'{c} - DISBURSED-DT'] = (df_bureau[c] - df_bureau['DISBURSED-DT']).dt.days
    df_bureau[f'{c} - DISBURSED-DT_max'] = (df_bureau[c] - df_bureau['DISBURSED-DT_max']).dt.days
    df_bureau[f'{c} - DISBURSED-DT_min'] = (df_bureau[c] - df_bureau['DISBURSED-DT_min']).dt.days
    


In [23]:
df_bureau[f'DATE-REPORTED - CLOSE-DT'] = (df_bureau['DATE-REPORTED'] - df_bureau['DISBURSED-DT']).dt.days
for f in [f'DATE-REPORTED - CLOSE-DT']:
    df[f] = df['ID'].map(df_bureau[fltr].set_index('ID')[f])

In [24]:

    
df_bureau['DISBURSED-AMT/HIGH CREDIT'] = df_bureau['DISBURSED-AMT/HIGH CREDIT'].apply(lambda x: clean(x))

In [25]:
fltr = df_bureau['app_dd'] == df_bureau['DISBURSED-DT']
df_bureau['disbursed_amt_diff'] = df_bureau.groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].shift(0) - df_bureau.groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].shift(-1)
for i in tqdm(range(-7, 7)):
    if i == 0:
        continue
    df_bureau[f'disbursed_amt_next_{i}'] = df_bureau.groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].shift(i)
    df_bureau[f'disbursed_amt_prev_{i}'] = df_bureau.groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].shift(-i)
    df_bureau[f'disbursed_amt_next_ACCT-TYPE_{i}'] = df_bureau.groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].shift(i)
    df_bureau[f'disbursed_amt_prev_ACCT-TYPE_{i}'] = df_bureau.groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].shift(-i)
    
    
    df_bureau[f'disbursed_amt_diff_to_next_{i}'] = df_bureau['DISBURSED-AMT/HIGH CREDIT'] - df_bureau[f'disbursed_amt_next_{i}']
    df_bureau[f'disbursed_amt_diff_to_prev_{i}'] = df_bureau['DISBURSED-AMT/HIGH CREDIT'] - df_bureau[f'disbursed_amt_prev_{i}']
    for f in [f'disbursed_amt_next_{i}', f'disbursed_amt_prev_{i}', f'disbursed_amt_diff_to_next_{i}',
              f'disbursed_amt_diff_to_prev_{i}', 'disbursed_amt_diff', f'disbursed_amt_next_ACCT-TYPE_{i}',
              f'disbursed_amt_prev_ACCT-TYPE_{i}']:
        
        df[f] = df['ID'].map(df_bureau[fltr].set_index('ID')[f])

In [26]:
fltr = df_bureau['app_dd'] == df_bureau['DISBURSED-DT']
df['ACCT-TYPE'] = df['ID'].map(df_bureau[fltr].set_index('ID')['ACCT-TYPE'])
df['ACCT-TYPE'] = pd.factorize(df['ACCT-TYPE'])[0]
df['TENURE'] = df['ID'].map(df_bureau[fltr].set_index('ID')['TENURE'])

In [27]:
fltr = (df_bureau['DISBURSED-DT'] == df_bureau['app_dd'])
df_bureau['app_close_dt'] = df_bureau['ID'].map(df_bureau[fltr].set_index('ID')['CLOSE-DT'])

fltr = df_bureau['DISBURSED-DT'] > df_bureau['app_dd']
df_bureau['tmp'] = (df_bureau['DISBURSED-DT'] < df_bureau['app_close_dt'])
df['n_loans_before_app_close_dt'] = df['ID'].map(df_bureau[fltr].groupby('ID')['tmp'].sum())

fltr = (df_bureau['DISBURSED-DT'] > df_bureau['app_dd']) & (df_bureau['DISBURSED-DT'] < df_bureau['app_close_dt'])
df['n_loans_after_curr_loan_before_app_close_dt'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].sum()).fillna(-1)

In [28]:
for c in ['CURRENT-BAL', 'INSTALLMENT-AMT', 'CREDIT-LIMIT/SANC AMT', 'ACCT-TYPE', 'DISBURSED-AMT/HIGH CREDIT', 'SELF-INDICATOR',
         'MATCH-TYPE', 'ba']:
    df[c + '_first'] = df['ID'].map(df_bureau.groupby('ID')[c].first())
    df[c + '_last'] = df['ID'].map(df_bureau.groupby('ID')[c].last())
    df[c + '_second_first'] = df['ID'].map(df_bureau.groupby('ID')[c].nth(1))
    df[c + '_second_last'] = df['ID'].map(df_bureau.groupby('ID')[c].nth(-2))
    df[c + '_third_first'] = df['ID'].map(df_bureau.groupby('ID')[c].nth(2))
    df[c + '_third_last'] = df['ID'].map(df_bureau.groupby('ID')[c].nth(-3))
    df[c + '_fourth_first'] = df['ID'].map(df_bureau.groupby('ID')[c].nth(3))
    df[c + '_fourth_last'] = df['ID'].map(df_bureau.groupby('ID')[c].nth(-4))
    df[c + '_fifth_first'] = df['ID'].map(df_bureau.groupby('ID')[c].nth(4))
    df[c + '_fifth_last'] = df['ID'].map(df_bureau.groupby('ID')[c].nth(-5))

In [29]:

df_bureau['tmp'] = df_bureau['DISBURSED-DT'] > df_bureau['app_dd']
df['n_loans_after_curr_loan'] = df['ID'].map(df_bureau.groupby('ID')['tmp'].sum())

df_bureau['tmp2'] = df_bureau['DISBURSED-AMT/HIGH CREDIT'] * df_bureau['tmp']
df['total_sum_disbursed_after_loan_applied'] =  df['ID'].map(df_bureau.groupby('ID')['tmp2'].sum())

df_bureau['tmp2'] = df_bureau['DISBURSED-AMT/HIGH CREDIT'] * (1 - df_bureau['tmp'])
df['total_sum_disbursed_before_loan_applied'] =  df['ID'].map(df_bureau.groupby('ID')['tmp2'].sum())

del df_bureau['tmp'], df_bureau['tmp2']
gc.collect()

df['mean_sum_disbursed_after_loan_applied'] = df['total_sum_disbursed_after_loan_applied']/df['n_loans_after_curr_loan']

In [30]:
fltr = df_bureau['app_dd'] == df_bureau['DISBURSED-DT']
df_bureau['total_loan_days'] = (df_bureau['CLOSE-DT'] - df_bureau['DISBURSED-DT']).dt.days
df['total_loan_days'] = df['ID'].map(df_bureau[fltr].set_index('ID')['total_loan_days'])
df['total_loan_days_max'] = df['ID'].map(df_bureau.groupby('ID')['total_loan_days'].max())
df['total_loan_days_min'] = df['ID'].map(df_bureau.groupby('ID')['total_loan_days'].min())
df['total_loan_days_first'] = df['ID'].map(df_bureau.groupby('ID')['total_loan_days'].first())
df['total_loan_days_last'] = df['ID'].map(df_bureau.groupby('ID')['total_loan_days'].last())
df['total_loan_days_range'] = df['total_loan_days_max'] - df['total_loan_days_min']

In [31]:
df['MaturityDAte']

0        2016-01-15
1        2016-02-15
2        2023-02-10
3        2017-11-10
4        2015-07-05
            ...    
143395   2015-02-10
143396   2015-04-05
143397   2015-04-05
143398   2014-08-10
143399   2014-08-10
Name: MaturityDAte, Length: 143400, dtype: datetime64[ns]

In [32]:
for c in ['DisbursalDate', 'MaturityDAte', 'AuthDate']:
    print(c)
    df[c + '_days_since_start'] = (df[c] - df[c].min()).dt.days
    df[c + '_days_from_end'] = (df[c].max() - df[c]).dt.days

df['DisbursalDate - MaturityDAte'] = (df['DisbursalDate'] - df['MaturityDAte']).dt.days

DisbursalDate
MaturityDAte
AuthDate


In [33]:
df.columns[:25]

Index(['ID', 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode',
       'BranchID', 'Area', 'Tenure', 'AssetCost', 'AmountFinance',
       'DisbursalAmount', 'EMI', 'DisbursalDate', 'MaturityDAte', 'AuthDate',
       'ManufacturerID', 'SupplierID', 'LTV', 'SEX', 'AGE', 'MonthlyIncome',
       'City', 'State', 'ZiPCODE', 'Top-up Month'],
      dtype='object')

In [34]:
for c in ['BranchID', 'Area', 'ManufacturerID', 'SupplierID', 'City', 'State', 'ZiPCODE']:
    df[c + 'dd_min'] = df.groupby(c)['DisbursalDate_days_since_start'].transform('max')
    df[c + '_dd_max'] = df.groupby(c)['DisbursalDate_days_since_start'].transform('min')

In [35]:
cat_cols = df.head().select_dtypes('object').columns.tolist()
cat_cols = [c for c in cat_cols if c not in ['DisbursalDate', 'MaturityDAte', 'AuthDate']]
print(cat_cols)
for c in cat_cols:
    df[c] = pd.factorize(df[c])[0]



['Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode', 'Area', 'SEX', 'City', 'State', 'CONTRIBUTOR-TYPE', 'ACCT-TYPE_first', 'ACCT-TYPE_last', 'ACCT-TYPE_second_first', 'ACCT-TYPE_second_last', 'ACCT-TYPE_third_first', 'ACCT-TYPE_third_last', 'ACCT-TYPE_fourth_first', 'ACCT-TYPE_fourth_last', 'ACCT-TYPE_fifth_first', 'ACCT-TYPE_fifth_last', 'SELF-INDICATOR_second_first', 'SELF-INDICATOR_second_last', 'SELF-INDICATOR_third_first', 'SELF-INDICATOR_third_last', 'SELF-INDICATOR_fourth_first', 'SELF-INDICATOR_fourth_last', 'SELF-INDICATOR_fifth_first', 'SELF-INDICATOR_fifth_last', 'MATCH-TYPE_first', 'MATCH-TYPE_last', 'MATCH-TYPE_second_first', 'MATCH-TYPE_second_last', 'MATCH-TYPE_third_first', 'MATCH-TYPE_third_last', 'MATCH-TYPE_fourth_first', 'MATCH-TYPE_fourth_last', 'MATCH-TYPE_fifth_first', 'MATCH-TYPE_fifth_last']


In [36]:
tmp = df_bureau[['ID', 'ACCT-TYPE']]
tmp = pd.get_dummies(tmp, columns = ['ACCT-TYPE'])
tmp = tmp.groupby('ID').sum().reset_index()
tmp.columns = ['ID'] + [c + '_total_for_user' for c in tmp.columns[1:]]
df = pd.merge(df, tmp, on = 'ID', how = 'left')

In [37]:
tmp = df_bureau[df_bureau['DISBURSED-DT'] > df_bureau['app_dd']][['ID', 'ACCT-TYPE']]
tmp = pd.get_dummies(tmp, columns = ['ACCT-TYPE'])
tmp = tmp.groupby('ID').sum().reset_index()
tmp.columns = ['ID'] + [c + '_total_for_user_2' for c in tmp.columns[1:]]
df = pd.merge(df, tmp, on = 'ID', how = 'left')

In [38]:
tmp = df_bureau[df_bureau['DISBURSED-DT'] > df_bureau['app_dd']]
for c in tqdm(tmp['CONTRIBUTOR-TYPE'].unique()):
    df[f'CONTRIBUTOR-TYPE_{c}_max_amount_after_loan'] = np.nan

In [39]:
tmp = df_bureau[df_bureau['DISBURSED-DT'] > df_bureau['app_dd']][['ID', 'ACCT-TYPE']]
for c in tqdm(tmp['ACCT-TYPE'].unique()):
    df[f'kharacha_{c}'] = df['ID'].map(df_bureau[df_bureau['ACCT-TYPE'] == c].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].sum())
    df[f'kharacha_{c}_max'] = np.nan

In [40]:
tmp = df_bureau[df_bureau['DISBURSED-DT'] > df_bureau['app_dd']]
for c in tqdm(tmp['ACCT-TYPE'].unique()):
    df[f'kharacha_2_{c}'] = df['ID'].map(tmp[tmp['ACCT-TYPE'] == c].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].sum())
    df[f'kharacha_2_{c}_max'] = df['ID'].map(tmp[tmp['ACCT-TYPE'] == c].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].max())
    df[f'kharacha_2_{c}_min'] = df['ID'].map(tmp[tmp['ACCT-TYPE'] == c].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].mean())
    df[f'kharacha_2_{c}_days_min'] = np.nan
#     df_bureau[]
#     df[f'kharacha_2_{c}_min'] = df['ID'].map(tmp[tmp['ACCT-TYPE'] == c].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].sum())
    df[f'kharcha_1_2_{c}_diff'] = df[f'kharacha_{c}'] - df[f'kharacha_2_{c}']

In [41]:
tmp = df_bureau[df_bureau['DISBURSED-DT'] <= df_bureau['app_dd']]
for c in tqdm(tmp['ACCT-TYPE'].unique()):
    df[f'kharacha_3_{c}'] = df['ID'].map(tmp[tmp['ACCT-TYPE'] == c].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].max())

In [42]:
df['days_diff_to_first_Disbursal_Date'] = (df['DisbursalDate'] - df['ID'].map(df_bureau.groupby('ID')['DISBURSED-DT'].min())).dt.days
df['days_diff_to_last_Disbursal_Date'] = (df['DisbursalDate'] - df['ID'].map(df_bureau.groupby('ID')['DISBURSED-DT'].max())).dt.days

In [43]:
fltr = df_bureau['DISBURSED-DT'] > df_bureau['app_dd']
df['CLOSING-DT-diff_to_next_loan'] = df['ID'].map((df_bureau[fltr].groupby('ID')['CLOSE-DT'].first() - df_bureau[fltr].groupby('ID')['CLOSE-DT'].nth(1)).dt.days)
df['CLOSING-DT-diff_to_next_loan']

fltr = df_bureau['DISBURSED-DT'] >= df_bureau['app_dd']
tmp = df_bureau[fltr]
tmp['cola'] = tmp.groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].transform('first') - df_bureau[fltr].groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].transform('nth', 1)
df['cola'] = tmp.groupby('ID')['cola'].first()
df['cola']

/home/sahil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0               NaN
1          130000.0
2           -4176.0
3           21630.0
4               NaN
            ...    
143395          NaN
143396     271176.0
143397   -1020000.0
143398   -1020000.0
143399    -641837.0
Name: cola, Length: 143400, dtype: float64

In [44]:
fltr = df_bureau['DISBURSED-DT'] >= df_bureau['app_dd']
tmp = df_bureau[fltr]
tmp['cola'] = tmp.groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].transform('first') - df_bureau[fltr].groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].transform('nth', 2)
df['cola2'] = tmp.groupby('ID')['cola'].first()


tmp['cola'] = tmp.groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].transform('first') - df_bureau[fltr].groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].transform('nth', 3)
df['cola3'] = tmp.groupby('ID')['cola'].first()

tmp['cola'] = tmp.groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].transform('first') - df_bureau[~fltr].groupby(['ID', 'ACCT-TYPE'])['DISBURSED-AMT/HIGH CREDIT'].transform('nth', -1)
df['cola4'] = tmp.groupby('ID')['cola'].first()

/home/sahil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/sahil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/sahil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [45]:
df_bureau['loan_closed'] = (df_bureau['LAST-PAYMENT-DATE'].isnull()) * 1
fltr = df_bureau['DISBURSED-DT'] <= df_bureau['app_dd']
df['loans_closed_before_disbursal_mean'] = df['ID'].map(df_bureau[fltr].groupby('ID')['loan_closed'].mean())

fltr = df_bureau['DISBURSED-DT'] > df_bureau['app_dd']
df['loans_closed_after_disbursal_mean'] = df['ID'].map(df_bureau[fltr].groupby('ID')['loan_closed'].mean())

In [46]:
df['TENURE'] = df['TENURE'].fillna(-1)
df['mean_TENURE'] = df['ID'].map(df_bureau.groupby('ID')['TENURE'].mean())
df['max_TENURE'] = df['ID'].map(df_bureau.groupby('ID')['TENURE'].max())
df['min_TENURE'] = df['ID'].map(df_bureau.groupby('ID')['TENURE'].min())
df['mean_TENURE']

0          60.000000
1          86.166667
2          66.000000
3          16.500000
4          46.400000
             ...    
143395    132.333333
143396           NaN
143397           NaN
143398           NaN
143399           NaN
Name: mean_TENURE, Length: 143400, dtype: float64

In [47]:
df['DisbursalDate_freq'] = df['DisbursalDate'].map(df['DisbursalDate'].value_counts())

In [48]:
df['mean_self_indicator'] = df['ID'].map(df_bureau.groupby('ID')['SELF-INDICATOR'].mean())
df['sum_self_indicator'] = df['ID'].map(df_bureau.groupby('ID')['SELF-INDICATOR'].sum())
fltr = df_bureau['DISBURSED-DT'] > df_bureau['app_dd']
tmp = df_bureau[fltr]
df['mean_self_indicator_after_loan'] = df['ID'].map(tmp.groupby('ID')['SELF-INDICATOR'].mean())
df['sum_self_indicator_after_loan'] = df['ID'].map(tmp.groupby('ID')['SELF-INDICATOR'].sum())

In [49]:
df_bureau['OWNERSHIP-IND'].unique()

array(['Individual', 'Guarantor', 'Joint', 'Primary', 'Supl Card Holder'],
      dtype=object)

In [50]:

for c in df_bureau['OWNERSHIP-IND'].unique():
    df_bureau['tmp'] = (df_bureau['OWNERSHIP-IND'] == c)*1
    df[f'mean_{c}_all'] = df['ID'].map(df_bureau.groupby('ID')['tmp'].mean())
    df[f'sum_{c}_all'] = df['ID'].map(df_bureau.groupby('ID')['tmp'].sum())

In [51]:

for c in df_bureau['OWNERSHIP-IND'].unique():
    df_bureau['tmp'] = (df_bureau['OWNERSHIP-IND'] == c)*1
    df[f'mean_{c}_all'] = df['ID'].map(df_bureau.groupby('ID')['tmp'].mean())
    df[f'sum_{c}_all'] = df['ID'].map(df_bureau.groupby('ID')['tmp'].sum())

In [52]:
for c in df_bureau['ACCOUNT-STATUS'].unique():
    df_bureau['tmp'] = (df_bureau['ACCOUNT-STATUS'] == c)*1
    df[f'mean_{c}_all'] = df['ID'].map(df_bureau.groupby('ID')['tmp'].mean())
    df[f'sum_{c}_all'] = df['ID'].map(df_bureau.groupby('ID')['tmp'].sum())

In [53]:
for a in tqdm(df_bureau['ACCT-TYPE'].unique()):
    for c in df_bureau['CONTRIBUTOR-TYPE'].unique():
        fltr = (df_bureau['ACCT-TYPE'] == a) & (df_bureau['CONTRIBUTOR-TYPE'] == c)
        if (fltr.sum()/fltr.shape[0]) > 0.2:
            df[f'sum_DISBURSED-AMT/HIGH CREDIT_{a}_{c}_all_accounts'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].sum())

In [54]:
fltr = df_bureau['DISBURSED-DT'] > df_bureau['app_dd']
tmp = df_bureau[fltr]
for c in df_bureau['SELF-INDICATOR'].unique():
    df_bureau['tmp'] = (df_bureau['SELF-INDICATOR'] == c)*1
    df[f'mean_{c}_all'] = df['ID'].map(tmp.groupby('ID')['tmp'].mean())
    df[f'sum_{c}_all'] = df['ID'].map(tmp.groupby('ID')['tmp'].sum())

In [55]:
fltr_a = df_bureau['DISBURSED-DT'] > df_bureau['app_dd']
tmp = df_bureau[fltr]
for c in df_bureau['SELF-INDICATOR'].unique():
    fltr_b = (df_bureau['SELF-INDICATOR'] == c)
    fltr_net = fltr_a & fltr_b
    df[f'mean_{c}_high_DISBURSED-AMT/HIGH CREDIT'] = df['ID'].map(df_bureau[fltr_net].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].mean())
    df[f'sum_{c}_DISBURSED-AMT/HIGH CREDIT'] = df['ID'].map(df_bureau[fltr_net].groupby('ID')['DISBURSED-AMT/HIGH CREDIT'].sum())

In [56]:
df = df.fillna(-9999)

In [57]:
for a in tqdm(df_bureau['ACCT-TYPE'].unique()):
    fltr = df_bureau['ACCT-TYPE'] == a
    df[f'ACCT-TYPE_{a}_DISBURSED_DT_max'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-DT_days_since_start'].max())
    df[f'ACCT-TYPE_{a}_DISBURSED_DT_min'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-DT_days_since_start'].min())
    df[f'ACCT-TYPE_{a}_DISBURSED_DT_range'] = (df[f'ACCT-TYPE_{a}_DISBURSED_DT_max'] -\
                                               df[f'ACCT-TYPE_{a}_DISBURSED_DT_min'])
    
    df['tmp'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-DT'].max())
    df[f'ACCT-TYPE_{a}_DISBURSED_DT_max - DisbursalDate'] = (df['tmp'] - df['DisbursalDate']).dt.days
    
del df['tmp']
gc.collect()

215

In [58]:
for a in tqdm(df_bureau['SELF-INDICATOR'].unique()):
    fltr = df_bureau['SELF-INDICATOR'] == a
    df[f'SELF-INDICATOR_{a}_DISBURSED_DT_max'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-DT_days_since_start'].max())
    df[f'SELF-INDICATOR_{a}_DISBURSED_DT_2nd_last'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-DT_days_since_start'].nth(-2))
    f = f'SELF-INDICATOR_{a}_DISBURSED_DT_max - SELF-INDICATOR_{a}_DISBURSED_DT_2nd_last'
    df[f] = df[f'SELF-INDICATOR_{a}_DISBURSED_DT_max'] - df[f'SELF-INDICATOR_{a}_DISBURSED_DT_2nd_last']
    del df[f'SELF-INDICATOR_{a}_DISBURSED_DT_2nd_last']
    df[f'SELF-INDICATOR_{a}_DISBURSED_DT_min'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-DT_days_since_start'].min())
    df[f'SELF-INDICATOR_{a}_DISBURSED_DT_range'] = (df[f'SELF-INDICATOR_{a}_DISBURSED_DT_max'] -\
                                               df[f'SELF-INDICATOR_{a}_DISBURSED_DT_min'])
    
    df['tmp'] = df['ID'].map(df_bureau[fltr].groupby('ID')['DISBURSED-DT'].max())
    df[f'SELF-INDICATOR_{a}_DISBURSED_DT_max - DisbursalDate'] = (df['tmp'] - df['DisbursalDate']).dt.days
    
del df['tmp']
gc.collect()

35

In [59]:
df = df.fillna(-1)

In [60]:
train, test = df[:train.shape[0]].reset_index(drop = True), df[train.shape[0]:].reset_index(drop = True)
train.shape, test.shape
threshold = 0.8
null_percent = (df.isnull().sum()/df.shape[0]).sort_values(ascending = False)
drop_fts_2 = null_percent[null_percent > threshold].index.tolist()

In [61]:
ignore_fts = ['DisbursalDate', 'MaturityDAte', 'AuthDate','last_reported_date', 'first_reported_date',
             'first_DISBURSED-AMT/HIGH CREDIT - DisbursalAmount','last_DISBURSED-AMT/HIGH CREDIT - DisbursalAmount',
             'n_loans_before_curr_loan', 'ACCT-TYPE_and_next_ACCT-TYPE_same', 'substandard_asset_class_count',
             'mean_days_number_of_loans_applied', 'total_sum_disbursed_after_loan_applied/day_diff_last',
             'total_sum_disbursed_after_loan_applied/DisbursalAmount', 'sum CUR BAL - HIST', 'mean_disbursed_amt_forward_diff',
             'Branch_ID_nunq_ACCT_TYPE', 'ACCT_TYPE_Branch_ID_nunq','next_ACCOUNT-STATUS',
              'ACCT-TYPE', 'last 3 mean CUR BAL - HIST',
           'DisbursalAmount/AssetCost', 'EMI/DisbursalAmount','last_DISBURSED-DT', 'first_DISBURSED-DT'
             ]

drop_fts = ['first_DISBURSED-AMT/HIGH CREDIT - last_DISBURSED-AMT/HIGH CREDIT',
 'ACCT-TYPE_Telco Landline_total_for_user',
 'ACCT-TYPE_Pradhan Mantri Awas Yojana - CLSS_total_for_user',
 'ACCT-TYPE_Business Non-Funded Credit Facility-Priority Sector-Others_total_for_user',
 'ACCT-TYPE_Commercial Equipment Loan_total_for_user',
 'ACCT-TYPE_Staff Loan_total_for_user',
 'ACCT-TYPE_Corporate Credit Card_total_for_user',
 'ACCT-TYPE_Prime Minister Jaan Dhan Yojana - Overdraft_total_for_user',
 'ACCT-TYPE_Fleet Card_total_for_user',
 'ACCT-TYPE_JLG Group_total_for_user',
 'ACCT-TYPE_SHG Individual_total_for_user',
 'ACCT-TYPE_Leasing_total_for_user',
 'ACCT-TYPE_Loan on Credit Card_total_for_user',
 'ACCT-TYPE_SHG Group_total_for_user',
 'ACCT-TYPE_Microfinance Personal Loan_total_for_user',
 'ACCT-TYPE_Microfinance Housing Loan_total_for_user']

fts = [c for c in train.columns if c not in [ID_COL,TARGET_COL] + ignore_fts + drop_fts + drop_fts_2]
print(len(fts))

1067


In [71]:
df[['ID']+fts].to_pickle('nikhil_feats.pkl')

In [558]:
np.random.seed(52)
N_SPLITS = 5
folds = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 2)
oofs = np.zeros((len(train), 7))
preds = np.zeros((len(test), 7))
fi_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train[TARGET_COL])):
    
    X_trn, y_trn = train[fts].iloc[trn_idx], train[TARGET_COL].iloc[trn_idx]
    X_val, y_val = train[fts].iloc[val_idx], train[TARGET_COL].iloc[val_idx]
    
    params = {
        'learning_rate': np.random.choice([0.05, 0.06, 0.08, 0.04, 0.1]),
        'colsample_bytree': np.random.choice([0.25, 0.5, 0.65, 0.8, 0.95, 0.7, 0.68]),
        'reg_alpha': np.random.choice([0.22, 0.85, 0.65, 0.7, 0.1]),
        'reg_lambda': np.random.choice([0.7, 0.1, 0.22, 0.85, 0.65, 0.05]),
        'max_depth': np.random.choice([4, 6, 8, 12, -1]),
        'n_estimators': 10000000000,
        'metric': 'None',
        'subsample_for_bin': 100000000000,
        'objective':'binary',
        'num_leaves': 127,
        'min_child_samples': 2,
    }

    print(params['max_depth'])
    
    clf = LGBMClassifier(**params)
    clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], verbose = 50, early_stopping_rounds = 100, eval_metric = 'multi_logloss')
    
    vp = clf.predict_proba(X_val)
    val_score = round(f1_score(y_val, vp.argmax(axis = 1), average = 'macro'), 5)
    
    print(f'\nFold {fold_+1} Val score: {val_score}\n')
    
    tp = clf.predict_proba(test[fts])
    
    oofs[val_idx] = vp
    preds += tp/N_SPLITS
        
    fi_df = fi_df.append(pd.DataFrame({'fold': fold_, 'feature': fts, 'importance': clf.feature_importances_}))
    
oof_score = round(f1_score(train[TARGET_COL], oofs.argmax(axis=1), average = 'macro'), 6)
print(f'\nOOF Auc is : {oof_score}')

8
Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_logloss: 0.280113
[100]	valid_0's multi_logloss: 0.282715
[150]	valid_0's multi_logloss: 0.290397
Early stopping, best iteration is:
[63]	valid_0's multi_logloss: 0.279803

Fold 1 Val score: 0.69817

12
Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_logloss: 0.292326
[100]	valid_0's multi_logloss: 0.27753
[150]	valid_0's multi_logloss: 0.277404
[200]	valid_0's multi_logloss: 0.280183
Early stopping, best iteration is:
[118]	valid_0's multi_logloss: 0.276869

Fold 2 Val score: 0.69266

6
Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_logloss: 0.291251
[100]	valid_0's multi_logloss: 0.281591
[150]	valid_0's multi_logloss: 0.279394
[200]	valid_0's multi_logloss: 0.279081
[250]	valid_0's multi_logloss: 0.279603
Early stopping, best iteration is:
[189]	valid_0's multi_logloss: 0.27883

Fold 3 Val score: 0.69414

-1
Training until valida

In [559]:
tmp = oofs.copy()
oof_score = f1_score(train[TARGET_COL], tmp.argmax(axis=1), average = 'macro')
oof_score

0.6986900805343287

In [560]:
tmp = oofs.copy()
tmp[tmp[:, 0] < 0.5, 0] = 0
# tmp[tmp[:, 6] < 0.15, 6] = 0
oof_score = f1_score(train[TARGET_COL], tmp.argmax(axis=1), average = 'macro')
f1_score(train[TARGET_COL], tmp.argmax(axis=1), average='macro')

0.7022080265181362

In [565]:
tmp = preds.copy()
tmp[tmp[:, 0] < 0.5, 0] = 0
pred_labels = pd.Series(tmp.argmax(axis = 1))
preds_actual = pred_labels.map(target_inv_mapper)

In [566]:
ss[TARGET_COL] = preds_actual
ss.to_csv('ltfs_3_rocks.csv', index=False)

In [567]:
ss[TARGET_COL].value_counts(normalize = True)

No Top-up Service    0.918142
 > 48 Months         0.037979
36-48 Months         0.014378
30-36 Months         0.008274
18-24 Months         0.007799
24-30 Months         0.007596
12-18 Months         0.005832
Name: Top-up Month, dtype: float64

In [568]:
ss[TARGET_COL].value_counts()

No Top-up Service    13538
 > 48 Months           560
36-48 Months           212
30-36 Months           122
18-24 Months           115
24-30 Months           112
12-18 Months            86
Name: Top-up Month, dtype: int64